In [3]:
has_pericardial_effusion.head()
Out[3]:

    survival  observed     age  pericardialeffusion  name
11      52.0       1.0  62.000                  1.0  name
15      24.0       1.0  55.000                  1.0  name
16       0.5       0.0  69.000                  1.0  name
17       0.5       0.0  62.529                  1.0  name
19       1.0       0.0  66.000                  1.0  name


In [4]:
none_pericardial_effusion.head()
Out[4]:

   survival  observed   age  pericardialeffusion  name
0      11.0       1.0  71.0                  0.0  name
1      19.0       1.0  72.0                  0.0  name
2      16.0       1.0  55.0                  0.0  name
3      57.0       1.0  60.0                  0.0  name
4      19.0       0.0  57.0                  0.0  name

In [ ]:
#packages are lifeline

In [ ]:
# Instantiate Kaplan Meier object for patients with and without pericardial effusion
kmf_has_pe = KaplanMeierFitter()
kmf_no_pe = KaplanMeierFitter()

# Fit Kaplan Meier estimators to each DataFrame
kmf_has_pe.fit(durations=has_pericardial_effusion['survival'], 
               event_observed=has_pericardial_effusion['observed'])
kmf_no_pe.fit(durations=none_pericardial_effusion['survival'], 
              event_observed=none_pericardial_effusion['observed'])

# Print out the median survival duration of each group
print("The median survival duration (months) of patients with pericardial effusion: ", kmf_has_pe.median_survival_time_)
print("The median survival duration (months) of patients without pericardial effusion: ", kmf_no_pe.median_survival_time_)

In [ ]:
The median survival duration (months) of patients with pericardial effusion:  27.0
The median survival duration (months) of patients without pericardial effusion:  31.0

#-- Now doing a Survival regression using AFT regressor 
-- WellBull is parametric way of things that is what the difference is 
-- if exp(beta-cofficint) > 1 => factor increass the survival time 
-- if exp(beta-cofficent) < 1 => factor decreases the survival time 


#### ------- BUSINESS INTERPRETATION ------------
Here’s a simple interpretation of each coefficient:

* Age (coef = 0.004, exp(coef) = 1.004)
Interpretation: Each additional year of age increases the expected survival duration by 0.4%. This suggests a very slight increase in survival time with age, possibly indicating that older individuals in this study have a marginally better survival prognosis under specific conditions.

* EPSS (coef = -0.022, exp(coef) = 0.978)
Interpretation: For each one-unit increase in EPSS, the survival duration decreases by about 2.2%. A higher EPSS score, which may reflect more severe heart conditions, is associated with shorter survival times.
* Fractional Shortening (coef = -0.120, exp(coef) = 0.887)
Interpretation: A unit decrease in fractional shortening (indicating worse heart function) is associated with a 11.3% reduction in survival time. This is a significant impact, emphasizing the importance of heart function in survival outcomes.

* LVDD (Left Ventricular Diastolic Diameter, coef = 0.066, exp(coef) = 1.068)
Interpretation: Each unit increase in LVDD is linked to a 6.8% increase in survival time. This might suggest that patients with a larger LVDD, under certain conditions, have longer survival times, although LVDD increases are typically a concern in heart health.

* Pericardial Effusion (coef = 0.026, exp(coef) = 1.026)
Interpretation: The presence of pericardial effusion increases survival time by 2.6%. This might indicate that in the context of other controlled factors, pericardial effusion does not significantly compromise survival, or may reflect the impact of successful management strategies.

* Wall Motion Index (coef = 0.131, exp(coef) = 1.140)
Interpretation: Each unit increase in the wall motion index, which might indicate more severe cardiac impairment, is associated with a 14% increase in survival time. This counterintuitive result could suggest compensatory mechanisms or other factors at play.
Wall Motion Score (coef = -0.010, exp(coef) = 0.990)
Interpretation: A higher wall motion score, potentially indicating more extensive heart damage, reduces survival time by 1% per unit increase.

** Intercept
Interpretation: This is a baseline multiplier for survival times when all other covariates are zero. It sets the fundamental scale of the survival time but isn't typically used for direct interpretation.

* Summary for Business:
Heart Function Parameters: Factors directly measuring heart function (EPSS, fractional shortening, LVDD) are crucial determinants of survival. Improvements or deteriorations in these measurements have direct, significant impacts on survival probabilities.
Age and Condition Specific Factors: Age slightly increases survival time, while specific conditions like pericardial effusion and changes in heart wall motion have nuanced impacts, which could vary depending on the patient's overall health and treatment strategies.
Management Implications: Monitoring and improving cardiac function should be a priority in patient management strategies to enhance survival outcomes.

## Custom- Model

Explore gender-LVDD interaction
You wonder if the survival risk for heart patients with LVDD (Left Ventricular Diastolic Dysfunction) is higher or lower among female patients. A good way to analyze this is with an interaction term between gender_f (1 if the subject is female, 0 otherwise) and lvdd.

You are going to fit the Weibull AFT model with gender_f*lvdd and interpret their coefficients. The DataFrame you will use is called heart_patients.

The pandas and numpy packages are loaded as pd and np, the WeibullAFTFitter class has been imported for you, and a WeibullAFTFitter instance aft has been created.

In [ ]:
# Fit custom model
aft.fit(heart_patients, 
        duration_col='survival', 
        event_col='observed',
        formula='epss + gender_f * lvdd')

# Print model summary
aft.summary

In [ ]:
<script.py> output:
                            coef  exp(coef)  se(coef)  coef lower 95%  coef upper 95%  ...  exp(coef) upper 95%  cmp to      z          p  -log2(p)
    param   covariate                                                                  ...                                                         
    lambda_ Intercept      3.419     30.545     0.492           2.454           4.384  ...               80.188     0.0  6.943  3.830e-12    37.926
            epss          -0.019      0.981     0.016          -0.052           0.013  ...                1.013     0.0 -1.184  2.364e-01     2.081
            gender_f      -0.064      0.938     1.151          -2.319           2.191  ...                8.941     0.0 -0.056  9.554e-01     0.066
            lvdd           0.084      1.088     0.110          -0.131           0.299  ...                1.349     0.0  0.764  4.450e-01     1.168
            gender_f:lvdd  0.017      1.018     0.245          -0.463           0.498  ...                1.646     0.0  0.071  9.432e-01     0.084
    rho_    Intercept      0.930      2.534     0.125           0.684           1.176  ...                3.240     0.0  7.413  1.233e-13    42.882
    

In [ ]:
## Suppose want to analyse the partial_plot function for a co-variate 
## means u can simulate the covariate different values to see the impact on survival model 


## Prediction on new_data 


In [ ]:
#Train_data & test data u have like same shape and u can use the same predict function to do that 


In [ ]:
# Predict median of new data
aft_pred = aft.predict_median(prison_new)

# Print average predicted time to arrest
print("On average, the median number of weeks for new released convicts to be arrested is: ", np.mean(aft_pred))

## Godness of Fit 
* Check the lowest aic score 
* check the qq_pot distrubtion as if its closer to the line , then the predictions are good  

In [ ]:
# Import qq_plot
from lifelines.plotting import qq_plot

# Plot qq_plot of wb
qq_plot(wb)

# Display figure
plt.show()

In [ ]:
In [1]:
heart_patients.head()
Out[1]:

   survival  observed
0      11.0       1.0
1      19.0       1.0
2      16.0       1.0
3      57.0       1.0
4      19.0       0.0

In [ ]:
# Instantiate each fitter
wb = WeibullFitter()
exp = ExponentialFitter()
log = LogNormalFitter()

# Fit to data
for model in [wb, exp, log]:
  model.fit(durations=heart_patients['survival'],
            event_observed=heart_patients['observed'])
  # Print AIC
  print(model.__class__.__name__, model.AIC_)

In [ ]:
script.py> output:
    WeibullFitter 331.95194834891294
    ExponentialFitter 367.02560610624914
    LogNormalFitter 334.83975916099 

# Cox-Ph 


In [ ]:
# Import CoxPHFitter class
from lifelines import CoxPHFitter

# Instantiate CoxPHFitter class cph
cph = CoxPHFitter()

# Fit cph to data
cph.fit(df=prison, duration_col="week", event_col="arrest")

# Print model summary
print(cph.summary)

In [ ]:
            coef  exp(coef)  se(coef)  coef lower 95%  coef upper 95%  ...  exp(coef) upper 95%  cmp to      z      p  -log2(p)
covariate                                                              ...                                                     
fin       -0.366      0.694     0.191          -0.740           0.009  ...                1.009     0.0 -1.915  0.056     4.171
age       -0.056      0.945     0.022          -0.099          -0.013  ...                0.987     0.0 -2.573  0.010     6.634
wexp      -0.157      0.855     0.212          -0.573           0.259  ...                1.295     0.0 -0.740  0.459     1.123
mar       -0.471      0.624     0.380          -1.217           0.274  ...                1.315     0.0 -1.239  0.215     2.216
paro      -0.078      0.925     0.195          -0.461           0.305  ...                1.356     0.0 -0.399  0.690     0.536
prio       0.090      1.094     0.029           0.033           0.146  ...                1.157     0.0  3.123  0.002     9.125

## Custome Model


In [ ]:
# Instantiate CoxPHFitter class
custom_cph = CoxPHFitter()

# Fit custom model
custom_cph.fit(df=prison, duration_col="week", event_col="arrest", formula="fin + age + prio")

# Print model summary
print(custom_cph.summary)

In [ ]:
print(custom_cph.summary)
            coef  exp(coef)  se(coef)  coef lower 95%  coef upper 95%  ...  exp(coef) upper 95%  cmp to      z          p  -log2(p)
covariate                                                              ...                                                         
age       -0.067      0.935     0.021          -0.108          -0.026  ...                0.974     0.0 -3.218  1.289e-03     9.599
fin       -0.347      0.707     0.190          -0.720           0.026  ...                1.026     0.0 -1.824  6.820e-02     3.874
prio       0.097      1.102     0.027           0.043           0.150  ...                1.162     0.0  3.555  3.776e-04    11.371


* **Hazard Ratio:** Direct interpretation from exp(coef). It tells us about the relative change in risk due to the covariate. A value less than 1 indicates a reduction in risk.


* **Surv_ratio:** Calculated to make the impact on survival time more intuitive. It inverses the hazard ratio to express how much longer individuals can survive due to the covariate, assuming a baseline survival without the covariate.

* In Cox Ph Model if **Surv_ratio:** This column, calculated as 1 / exp(coef), provides an intuitive measure of the impact of each covariate on survival. A ratio greater than 1 suggests a protective effect (enhancing survival), while a ratio less than 1 suggests a detrimental effect (reducing survival).

* **How to Interpret a Coef in COX PH**
> Hazard Ratio (0.694): This value indicates that financial assistance is associated with a hazard that is 69.4% of the hazard for those without financial assistance. To find the percentage reduction in hazard:
1
−
0.694
=
0.306
1−0.694=0.306
This result means there is a 30.6% reduction in the hazard due to financial assistance.

* **How to Interpret the Survival Ratio**
> Surv_ratio (1.441) implies that if the survival time without financial assistance is considered 100%, the survival time with financial assistance extends to 144.1% of the baseline (100%). The additional survival time can be calculated as:
144.1
−
100
=
44.1
%
144.1−100=44.1%
Thus, having financial assistance effectively increases the survival duration by 44.1% compared to not having financial assistance, assuming all other factors are held constant.

In [ ]:
# Instantiate CoxPHFitter class
cph = CoxPHFitter()

# Fit cph to data using all columns
cph.fit(df=prison, duration_col="week", event_col="arrest")

# Assign summary to summary_df
summary_df = cph.summary

# Create new column of survival time ratios
summary_df["surv_ratio"] = 1 / summary_df["exp(coef)"]

# Print surv_ratio for prio
print(summary_df.loc["prio", "surv_ratio"])

In [ ]:
## PLOT THE different covariate RANGE OF VALUES ON SURVIVALFUNCTIONS 
# Plot partial effects
cph.plot_partial_effects_on_outcome(covariates=["fin", "age", "wexp", "mar", "paro", "prio"],
                                   values=[[0, 35, 0, 1, 1, 3],
                                           [1, 22, 0, 0, 0, 0]])

# Show plot
plt.show()

## Check Assumption 
** Good job! Despite being a visual and seemingly relaxed check, Kaplan-Meier curves that seem parallel indicate the proportional hazards assumption is satisfied.

In [ ]:
# Fit to fin == 0
kmf.fit(durations=prison[prison["fin"] == 0]["week"], 
        event_observed=prison[prison["fin"] == 0]["arrest"])

# Plot survival curve for fin == 0
ax = kmf.plot()

# Fit to fin == 1 and plot on ax
kmf.fit(durations=prison[prison["fin"] == 1]["week"], 
        event_observed=prison[prison["fin"] == 1]["arrest"])
kmf.plot(ax=ax)
plt.show()

In [ ]:
print(cph.check_assumptions(training_df=prison, p_value_threshold=0.1))

In [ ]:
# Check PH assumption
print(cph.check_assumptions(training_df=prison, p_value_threshold=0.1))
The ``p_value_threshold`` is set at 0.1. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 432 total observations, 318 right-censored observations>
         test_name = proportional_hazard_test

---
           test_statistic      p  -log2(p)
age  km             11.51 <0.005     10.50
     rank           11.93 <0.005     10.83
fin  km              0.00   0.95      0.08
     rank            0.00   0.96      0.06
mar  km              0.78   0.38      1.41
     rank            0.90   0.34      1.54
paro km              0.16   0.69      0.54
     rank            0.18   0.67      0.58
prio km              0.02   0.89      0.16
     rank            0.01   0.90      0.15
wexp km              7.86   0.01      7.63
     rank            7.70   0.01      7.50


1. Variable 'age' failed the non-proportional test: p-value is 0.0006.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'wexp' failed the non-proportional test: p-value is 0.0050.

   Advice: with so few unique values (only 2), you can include `strata=['wexp', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification

[]

## Case Study- Employee Churn 


**Employee churn study**
Acquiring new employees as replacements incurs hiring costs and training costs. You want to predict how long current employees will stay. This exercise focuses on the first steps to prepare to make predictions.

You have a DataFrame called employees. It contains data about 1470 employees (churned and present) and how their survey answers. The survey is across the following dimensions:

* environment_satisfaction
* job_satisfaction
* relationship_satisfaction
* work_life_balance
Additionally, years_at_company means the duration employees have worked and attrition indicates if the employee has churned (1 if churn, 0 otherwise).

Sample rows are printed for you in the console. The CoxPHFitter class is imported for you from the lifelines package.

In [ ]:
TRAINING-DATA 

   attrition  years_at_company  environment_satisfaction  job_satisfaction  relationship_satisfaction  work_life_balance  distance_from_home  performance_rating  hourly_rate  stock_option_level
0          1                 6                         2                 4                          1                  1                   1                   3           94                   0
1          0                10                         3                 2                          4                  3                   8                   4           61                   1

In [ ]:
# Instantiate a CoxPHFitter object cph
cph = CoxPHFitter()

# Fit cph on all covariates
cph.fit(employees, duration_col='years_at_company', event_col='attrition')

# Select employees that have not churned
current_employees = employees.loc[employees['attrition'] == 0] ## 

# Existing durations of employees that have not churned
current_employees_last_obs = current_employees['years_at_company']

In [ ]:

TEST-DATA/ CURRENT-EMPLOYEE_BASE WHO ARE IN SYSTEM 


   Attrition  YearsAtCompany  EnvironmentSatisfaction  JobSatisfaction  RelationshipSatisfaction  WorkLifeBalance  DistanceFromHome  PerformanceRating  HourlyRate  StockOptionLevel
1          0              10                        3                2                         4                3                 8                  4          61                 1
3          0               8                        4                3                         3                3                 3                  3          56                 0
4          0               2                        1                2                         4                3                 2                  3          40                 1
5          0               7                        4                4                         3                2                 2                  3          79                 0
6          0               1                        3                1                         1                2                 3                  4          81                 3

In [ ]:
# Predict survival function conditional on existing durations
cph.predict_survival_function(current_employees, 
                              conditional_after=current_employees_last_obs)

In [ ]:
# Predict survival function conditional on existing durations
cph.predict_survival_function(current_employees, 
                              conditional_after=current_employees_last_obs)

# Predict median remaining times for current employees
pred = cph.predict_median(current_employees, 
                          conditional_after=current_employees_last_obs)

# Print the smallest median remaining time
print(min(pred))